In [87]:
import cv2
import sys
import pandas as pd
import numpy as np
import os
import pytesseract
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [101]:
def imagePreprocess(img):
    # Rescale the image, if needed.
    img = cv2.resize(img, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_CUBIC)
    # Convert to gray
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply dilation and erosion to remove some noise
    kernel = np.ones((1, 1), np.uint8)
    img = cv2.dilate(img, kernel, iterations=1)
    img = cv2.erode(img, kernel, iterations=1)
    # Apply threshold to get image with only black and white
    img = cv2.adaptiveThreshold(cv2.medianBlur(img, 3), 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2)
    
    return img

In [88]:
#IMAGE TO TEXT METHOD
def image_to_text(directoryList):
    print("Reading images.......!!!!")
    dataFile=[]
    exclude=set(string.punctuation)
    for imPath in directoryList:
        
        print(imPath)
        # Define config parameters.
        # '-l eng'  for using the English language 
        # '--oem 1' for using LSTM OCR Engine
        config = ('-l eng --oem 1 --psm 3')
    
        # Read image from disk
        img = cv2.imread(str(imPath), cv2.IMREAD_COLOR)
        
        # Apply preprocessing on image
        im = imagePreprocess(img)
        # Run tesseract OCR on image
        text = pytesseract.image_to_string(im, config=config)
        #basic preprocessing of the text
        text = text.replace('\n',' ')
        text = text.replace('\t',' ')
        text = text.replace('\n+',' ')
        text = text.replace('\t+',' ')
        text= text.rstrip()
        text= text.lstrip()
        text = text.replace(' +',' ')
        text= ''.join(char for char in text if char not in exclude)
        #text = text.lower()
        dataFile.append(text)
    return dataFile

In [ ]:
import re
imPath='/home/ikscare/Documents/Projects/Mousam/imageFile/Boon Chapman (4).txt-page0.tiff'
#imPath='/home/ikscare/Documents/Projects/Mousam/imageFile/BCBS Correspondence (2).txt-page0.tiff'
im = cv2.imread(str(imPath), cv2.IMREAD_COLOR)
#hocr=pytesseract.image_to_pdf_or_hocr(im, extension='hocr')
newdata=pytesseract.image_to_osd(im)
re.search('(?<=Rotate: )\d+', newdata).group(0)
print(newdata)

In [ ]:
from scipy import ndimage

#rotation angle in degree
rotated = ndimage.rotate(im, 360-int(re.search('(?<=Rotate: )\d+', pytesseract.image_to_osd(im)).group(0)))

In [ ]:
print(pytesseract.image_to_osd(rotated))

In [89]:
testImg="/home/ikscare/Documents/Projects/Mousam/DC_using_OCR/TestImages"
images = []

for filename in os.listdir(testImg):
	img = os.path.join(testImg,filename)
	if img is not None:
		images.append(img)
#print(images)
#readContent=images_to_text(images)
readContent=image_to_text(images)
print("Done")
print('Total number of images read and converted: ',len(readContent))

Reading images.......!!!!
/home/ikscare/Documents/Projects/Mousam/DC_using_OCR/TestImages/947137773e4b7ee45ce00e1d289e08b9.jpg
/home/ikscare/Documents/Projects/Mousam/DC_using_OCR/TestImages/Home-Health-Care-Hospice-768x995.jpg
/home/ikscare/Documents/Projects/Mousam/DC_using_OCR/TestImages/grady-memorial-hospital-research-oversight-roc-applicationdoc-1-638.jpg
/home/ikscare/Documents/Projects/Mousam/DC_using_OCR/TestImages/bca8a3ee2030590d13f5e723d339e626 (1).png
/home/ikscare/Documents/Projects/Mousam/DC_using_OCR/TestImages/Blood-report-of-patient-showing-Protein-S-deficiency.png
/home/ikscare/Documents/Projects/Mousam/DC_using_OCR/TestImages/Children-Medical.jpg
/home/ikscare/Documents/Projects/Mousam/DC_using_OCR/TestImages/free-personal-health-record-template-good-design-67-medical-history-forms-word-pdf-printable-templates.jpg
/home/ikscare/Documents/Projects/Mousam/DC_using_OCR/TestImages/DB777.jpg
/home/ikscare/Documents/Projects/Mousam/DC_using_OCR/TestImages/Patient-Medical-

In [56]:
#model building process
directory='/home/ikscare/Documents/Projects/Mousam/DC_using_OCR/DocKeyword.xls'
docKywrdmppr=pd.read_excel(directory)
docKywrdmppr

,Document Type,Keywords
0,Consult Notes,Consultation consultants Physical Exam Previo...
1,Advance Directives,Directive To physician Resuscitate Medical Pow...
2,Cardiopulmonary,ECG Carotid Doppler Stress Test Echo Rhythm Tr...
3,Care Management,Goal Barrier Care Plan Start Date End Date Con...
4,Chart Notes,Questionnaire Scale Clock Tele communication T...
5,Colonoscopy,Colonoscopy Sigmoidoscopy Large Bowel Polyps D...
6,Consents,Consent Tuberculin Records Release Disclosure ...
7,Correspondence,drug interactions Nonadherence therapy medical...
8,HIPAA General Consent,HIPAA Records Release General Consent
9,Home Health Hospice,Physician order Physician verbal order Recerti...


In [66]:
docKywrdmppr['Keywords'][0]
len(docKywrdmppr)

25

In [90]:
catAccum=[]
for cat in range(len(docKywrdmppr)):
    catAccum.append(0)

In [91]:
predictedDoc=[]
counter=1
#readContent=['Exam Date Technique/Technician Findings',"DL LA I am A DS Insurance Driver's License"]

In [ ]:
#readContent=['Diabetes Eye Exam Referral and Communication Form = .:1oy     To: OphaimologstTOptometist RON Refeing Psa Primary Care Powder Name: Name  eres  Phone     Fax:        REFERRAL INFORMATION     Primary Care Provider: Please complete section below. have refered the following patient to you for a diated retinal examination for evaluation of diabetic retinopathy.  Patient Name/ID#: oe:     Insurance:  Date of Scheduled Eye Exam:     EYE EXAMINATION REPORT     Eye Care Specialist: Please fax or mail this form tothe Primary Care Provider listed above upon completion ofthe patient vst. Please include your treatment plan,  Retinal Examination Findings:  Cy No diabetic retinopathy  Gi Retinal normalities detected, as folows: Gi Non-proiferative changes noted in:  Right (Grade) CWA C] Mid E)Moderate C1 severe ‘lnicaly significant diabetic macular edema? LJ YES [NO  Lett (Grade) CWA Cl mid Moderate 1] severe ‘Clinically signincant diabetic macular edema']

In [102]:
# predictor algorithm
for docs in readContent:
    for x in range(len(docKywrdmppr)):
        catAccum[x]=0
    for i in range(len(docKywrdmppr)):
        print('------------In Document %s--------------'% docKywrdmppr['Document Type'][i])
        for word in removeStopWords(docs):
            if word.casefold() in removeStopWords(docKywrdmppr['Keywords'][i].casefold()):
                print(word)
                catAccum[i]=catAccum[i]+counter
           # for key in docKywrdmppr['Keywords'][i]:
           #     if word.casefold() == key.casefold():
            #        print(word)
            #        #print('i=',i) 
             #       #print(catAccum[i])
             #       catAccum[i]=catAccum[i]+counter
    print(catAccum)
    ind=catAccum.index(max(catAccum))
    print(ind)
    predictedDoc.append(docKywrdmppr['Document Type'][ind])

Date
information
Code
Form
Date
[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0]
3
Diagnosis
Surgery
Physician
PHYSICIAN
Care
Date
Physician
PHYSICIAN
Home
Health
Hospice
Emergency
Home
Care
Immunization
received
Last
Orders
Care
Physician
Primary
PHYSICIAN
ORDERS
Surgery
Date
Physician
PHYSICIAN
referral
REFERRAL
Medicaid
[2, 2, 0, 2, 0, 0, 0, 0, 0, 5, 2, 0, 1, 1, 0, 1, 0, 1, 7, 0, 1, 0, 4, 1, 0]
18
System
Plan
Chief
Date
Plan
telephone
Authorization
Information
Consent
HIPAA
Consent
Health
Department
Insurance
Authorization
Request
Code
form
Form
Date
Clinical
approved
Authorization
Approved
Payment
Study
[3, 0, 0, 2, 1, 0, 3, 0, 2, 1, 1, 0, 0, 0, 1, 0, 0, 1, 2, 2, 2, 0, 3, 1, 1]
0
INFORMATION
EMERGENCY
INSURANCE
FORM
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
6
Diagnosis
test
Details
changes
Date
retinal
Range
Received
normal
Results
Date
Impression
Necessary
[1, 0, 3, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 3, 0, 0, 0, 0, 3, 0

In [93]:
predictedDoc

['Care Management',
 'Orders',
 'Consult Notes',
 'Consents',
 'Cardiopulmonary',
 'Consult Notes',
 'Consult Notes',
 'Care Management',
 'Consult Notes',
 'Lab',
 'Consents',
 'Radiology',
 'Consult Notes',
 'Medication Requests',
 'Orders',
 'Care Management',
 'Orders']

In [ ]:
# initial method for prediction
for docs in readContent:
    for x in range(len(docKywrdmppr)):
        catAccum[x]=0
    for word in docs:
        totalOccurances=0
        for x in range(len(docKywrdmppr)):
            if word.lower() in docKywrdmppr['Keywords'][x].lower():
                totalOccurances=totalOccurances+1
                catAccum[x]=catAccum[x]+1
    for x in range(len(docKywrdmppr)):
        catAccum[x]=catAccum[x]/totalOccurances
    print(catAccum.index(max(catAccum)))

In [103]:
# predictor algorithm 2 
for docs in readContent:
    for x in range(len(docKywrdmppr)):
        catAccum[x]=0
    for i in range(len(docKywrdmppr)):
        print('------------In Document %s--------------'% docKywrdmppr['Document Type'][i])
        for word in removeStopWords(docs):
            totalOccurances=1
            if word.casefold() in removeStopWords(docKywrdmppr['Keywords'][i].casefold()):
                print(word)
                totalOccurances=totalOccurances+1
                catAccum[i]=catAccum[i]+counter
           # for key in docKywrdmppr['Keywords'][i]:
           #     if word.casefold() == key.casefold():
            #        print(word)
            #        #print('i=',i) 
             #       #print(catAccum[i])
             #       catAccum[i]=catAccum[i]+counter
            
    for x in range(len(docKywrdmppr)):
        catAccum[x]=catAccum[x]/totalOccurances
    print('Accumulated list')
    print(catAccum)
    ind=catAccum.index(max(catAccum))
    print(ind)
    predictedDoc.append(docKywrdmppr['Document Type'][ind])
    
predictedDoc

------------In Document Consult Notes--------------
------------In Document Advance Directives--------------
------------In Document Cardiopulmonary--------------
------------In Document Care Management--------------
Date
------------In Document Chart Notes--------------
------------In Document Colonoscopy--------------
------------In Document Consents--------------
information
------------In Document Correspondence--------------
------------In Document HIPAA General Consent--------------
------------In Document Home Health Hospice--------------
------------In Document Hospital Records--------------
------------In Document In-office Diagnostics--------------
------------In Document Urgent care--------------
------------In Document Immunizations--------------
------------In Document Insurance DL--------------
------------In Document Lab--------------
------------In Document Mammography--------------
------------In Document Medication Requests--------------
------------In Document Orders

------------In Document Consents--------------
------------In Document Correspondence--------------
Medical
------------In Document HIPAA General Consent--------------
------------In Document Home Health Hospice--------------
physician
health
------------In Document Hospital Records--------------
medication
------------In Document In-office Diagnostics--------------
------------In Document Urgent care--------------
------------In Document Immunizations--------------
History
history
given
------------In Document Insurance DL--------------
------------In Document Lab--------------
------------In Document Mammography--------------
------------In Document Medication Requests--------------
medication
------------In Document Orders--------------
physician
Medical
------------In Document Patient info Forms--------------
------------In Document Radiology--------------
History
history
------------In Document Upper Endoscopy--------------
------------In Document Referrals--------------
approved


RELEASE
General
------------In Document Home Health Hospice--------------
------------In Document Hospital Records--------------
------------In Document In-office Diagnostics--------------
------------In Document Urgent care--------------
------------In Document Immunizations--------------
given
------------In Document Insurance DL--------------
------------In Document Lab--------------
received
------------In Document Mammography--------------
------------In Document Medication Requests--------------
AUTHORIZATION
authorization
------------In Document Orders--------------
------------In Document Patient info Forms--------------
RELEASE
------------In Document Radiology--------------
results
date
Date
------------In Document Upper Endoscopy--------------
------------In Document Referrals--------------
AUTHORIZATION
authorization
------------In Document Waivers--------------
------------In Document Sleep Studies--------------
Accumulated list
[1.0, 0.0, 0.0, 2.0, 1.0, 0.0, 4.0, 0.0, 2.0

------------In Document Referrals--------------
------------In Document Waivers--------------
------------In Document Sleep Studies--------------
Accumulated list
[1.0, 1.0, 1.0, 2.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
3
------------In Document Consult Notes--------------
Return
MEDICAL
return
------------In Document Advance Directives--------------
MEDICAL
PHYSICIAN
------------In Document Cardiopulmonary--------------
------------In Document Care Management--------------
------------In Document Chart Notes--------------
Telephone
------------In Document Colonoscopy--------------
------------In Document Consents--------------
------------In Document Correspondence--------------
MEDICAL
------------In Document HIPAA General Consent--------------
------------In Document Home Health Hospice--------------
HEALTH
PHYSICIAN
------------In Document Hospital Records--------------
instructions
------------In Document In-offi

['Care Management',
 'Orders',
 'Consult Notes',
 'Consents',
 'Cardiopulmonary',
 'Consult Notes',
 'Consult Notes',
 'Care Management',
 'Consult Notes',
 'Lab',
 'Consents',
 'Radiology',
 'Consult Notes',
 'Medication Requests',
 'Orders',
 'Care Management',
 'Orders',
 'Care Management',
 'Orders',
 'Consult Notes',
 'Consents',
 'Cardiopulmonary',
 'Consult Notes',
 'Consult Notes',
 'Care Management',
 'Consult Notes',
 'Lab',
 'Consents',
 'Radiology',
 'Consult Notes',
 'Medication Requests',
 'Orders',
 'Care Management',
 'Orders',
 'Care Management',
 'Orders',
 'Consult Notes',
 'Consents',
 'Cardiopulmonary',
 'Consult Notes',
 'Consult Notes',
 'Care Management',
 'Consult Notes',
 'Lab',
 'Consents',
 'Radiology',
 'Consult Notes',
 'Medication Requests',
 'Orders',
 'Care Management',
 'Orders',
 'Care Management',
 'Orders',
 'Consult Notes',
 'Consents',
 'Cardiopulmonary',
 'Consult Notes',
 'Consult Notes',
 'Care Management',
 'Consult Notes',
 'Lab',
 'Consents'

In [100]:
# see predicted data
i=0
for data in readContent:
    print('\n\n\nDocument ',i+1)
    print(data,"\n\nPredicted as:",predictedDoc[i])
    i=i+1




Document  1
Company Name INVOICE  Company Address Line Here  City State Zip Code  Phone xxxxxxxxxx  Fax xxXXXKK  Website wwwcompanywebsitecom     Name Date Addres  City By Zip Code Phone  Allt Phone        Shipping information Form     City Description                                                           Important Notes  Sub Total Tax Total Due  Authorized Signatures     Thank You for Your Business 

Predicted as: Care Management



Document  2
REFERRAL PHYSICIAN ORDERS  Home Health Care and Hospice  ©                          STMA RY’S Direct Physician Line 706 3892255 SETAE TERRES POU Direct Fax to Intake 706 3892298 Name SS AddressService Location Zip County Phone DOB Male Female Immunization Status Emergency Contact Relationship Emergency Contact Number  Medicare CO Medicaid  0 Private Insurani Policy        Home Care Services  Hospice Services                          RNG sLP a Msw 2 Home Hospice C1  oT O AIDE O Telehealth O  pT Hospice House Inpatient C Weight Bearing Sta

In [ ]:
predictedDoc

In [ ]:
for i in range(len(docKywrdmppr)):
    print(catAccum[i])

In [ ]:
catAccum[1]

In [73]:
def removeStopWords(docs):
    stop_words = set(stopwords.words('english')) 
    #for docs in readContent:
    word_tokens = word_tokenize(docs)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    filtered_sentence = [] 
    for w in word_tokens: 
        if w not in stop_words:
            filtered_sentence.append(w)
    filt_set=set(filtered_sentence)
    filtered_sentence=list(filt_set)
    return filtered_sentence

In [24]:
db=removeStopWords('Hey this is Mousam Singh')
db
for d in db:
    print(d)

Hey
Mousam
Singh
